# Coordinate Alignment

Since linopy builds on xarray, coordinate alignment matters when combining variables or expressions that live on different coordinates. By default, linopy aligns operands automatically and fills missing entries with sensible defaults. This guide shows how alignment works and how to control it with the ``join`` parameter.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import linopy

## Default Alignment Behavior

All arithmetic operations (``+``, ``-``, ``*``, ``/``) use **"outer"** alignment by default — the union of coordinates. Missing positions are filled with operation-appropriate defaults: zero for addition and subtraction, zero coefficients for multiplication.

In [ ]:
m = linopy.Model()

time = pd.RangeIndex(5, name="time")
x = m.add_variables(lower=0, coords=[time], name="x")

subset_time = pd.RangeIndex(3, name="time")
y = m.add_variables(lower=0, coords=[subset_time], name="y")

Adding ``x`` (5 time steps) and ``y`` (3 time steps) gives an expression over all 5 time steps. Where ``y`` has no entry (time 3, 4), the coefficient is zero — i.e. ``y`` simply drops out of the sum at those positions.

In [ ]:
x + y

The same applies when multiplying by a constant that covers only a subset of coordinates. Missing positions get a coefficient of zero:

In [ ]:
factor = xr.DataArray([2, 3, 4], dims=["time"], coords={"time": [0, 1, 2]})
x * factor

Adding a constant subset also fills missing coordinates with zero:

In [ ]:
x + factor

### Constraints with Subset RHS

For constraints, missing right-hand-side values are filled with ``NaN``, which tells linopy to **skip** the constraint at those positions:

In [ ]:
rhs = xr.DataArray([10, 20, 30], dims=["time"], coords={"time": [0, 1, 2]})
con = x <= rhs
con

The constraint only applies at time 0, 1, 2. At time 3 and 4 the RHS is ``NaN``, so no constraint is created.

### Same-Shape Operands with Different Labels

When two operands have the same shape but different coordinate labels, the default ``"outer"`` alignment produces the **union** of all coordinates. Each operand only contributes at its own positions:

In [ ]:
z = m.add_variables(lower=0, coords=[pd.RangeIndex(5, 10, name="time")], name="z")
x + z

``x`` (time 0–4) and ``z`` (time 5–9) share no coordinate labels, so the result spans all 10 time steps — each variable appears only at its own positions.

The same applies to constant operands with different labels:

In [ ]:
offset_const = xr.DataArray(
    [10, 20, 30, 40, 50], dims=["time"], coords={"time": [5, 6, 7, 8, 9]}
)
x + offset_const

The result spans 10 time steps. At time 0–4 only ``x`` contributes; at time 5–9 only the constant applies.

To force **positional** alignment (ignoring labels), use ``join="override"``:

In [ ]:
x.add(z, join="override")

With ``join="override"``, positions are matched directly and the left operand's labels are used. This can be useful when coordinate labels are irrelevant and you just want to pair entries by position.

## The ``join`` Parameter

For explicit control over alignment, use the ``.add()``, ``.sub()``, ``.mul()``, and ``.div()`` methods with a ``join`` parameter. The supported values follow xarray conventions:

- ``"inner"`` — intersection of coordinates
- ``"outer"`` — union of coordinates (with fill)
- ``"left"`` — keep left operand's coordinates
- ``"right"`` — keep right operand's coordinates
- ``"override"`` — positional alignment, ignore coordinate labels
- ``"exact"`` — coordinates must match exactly (raises on mismatch)

In [ ]:
m2 = linopy.Model()

i_a = pd.Index([0, 1, 2], name="i")
i_b = pd.Index([1, 2, 3], name="i")

a = m2.add_variables(coords=[i_a], name="a")
b = m2.add_variables(coords=[i_b], name="b")

**Inner join** — only shared coordinates (i=1, 2):

In [ ]:
a.add(b, join="inner")

**Outer join** — union of coordinates (i=0, 1, 2, 3):

In [ ]:
a.add(b, join="outer")

**Left join** — keep left operand's coordinates (i=0, 1, 2):

In [ ]:
a.add(b, join="left")

**Right join** — keep right operand's coordinates (i=1, 2, 3):

In [ ]:
a.add(b, join="right")

**Override** — positional alignment, ignore coordinate labels. The result uses the left operand's coordinates. Here ``a`` has i=[0, 1, 2] and ``b`` has i=[1, 2, 3], so positions are matched as 0↔1, 1↔2, 2↔3:

In [ ]:
a.add(b, join="override")

### Multiplication with ``join``

The same ``join`` parameter works on ``.mul()`` and ``.div()``. When multiplying by a constant that covers a subset, ``join="inner"`` restricts the result to shared coordinates only, while ``join="left"`` fills missing values with zero:

In [ ]:
const = xr.DataArray([2, 3, 4], dims=["i"], coords={"i": [1, 2, 3]})

a.mul(const, join="inner")

In [ ]:
a.mul(const, join="left")

## Alignment in Constraints

For constraints with a DataArray right-hand side, the default alignment is ``"left"`` — the expression defines where constraints exist, and missing RHS values become ``NaN`` (meaning "no constraint").

The ``.le()``, ``.ge()``, and ``.eq()`` methods accept the same ``join`` parameter for explicit control:

In [ ]:
rhs = xr.DataArray([10, 20], dims=["i"], coords={"i": [0, 1]})

a.le(rhs, join="inner")

With ``join="inner"``, the constraint only exists at the intersection (i=0, 1). Compare with the default ``join="left"``:

In [ ]:
a.le(rhs, join="left")

With ``join="left"`` (the default for constraints with DataArray RHS), the result covers all of ``a``'s coordinates (i=0, 1, 2). At i=2, where the RHS has no value, the RHS becomes ``NaN`` and the constraint is masked out.

The same methods work on expressions:

In [ ]:
expr = 2 * a + 1
expr.eq(rhs, join="inner")

## Practical Example

Consider a generation dispatch model where solar availability follows a daily profile and a minimum demand constraint only applies during peak hours.

In [ ]:
m3 = linopy.Model()

hours = pd.RangeIndex(24, name="hour")
techs = pd.Index(["solar", "wind", "gas"], name="tech")

gen = m3.add_variables(lower=0, coords=[hours, techs], name="gen")

Capacity limits apply to all hours and techs — standard broadcasting handles this:

In [ ]:
capacity = xr.DataArray([100, 80, 50], dims=["tech"], coords={"tech": techs})
m3.add_constraints(gen <= capacity, name="capacity_limit")

For solar, we build a full 24-hour availability profile — zero at night, sine-shaped during daylight (hours 6–18). Since this covers all hours, standard alignment works directly and solar is properly constrained to zero at night:

In [ ]:
solar_avail = np.zeros(24)
solar_avail[6:19] = 100 * np.sin(np.linspace(0, np.pi, 13))
solar_availability = xr.DataArray(solar_avail, dims=["hour"], coords={"hour": hours})

solar_gen = gen.sel(tech="solar")
m3.add_constraints(solar_gen <= solar_availability, name="solar_avail")

Now suppose a minimum demand of 120 MW must be met, but only during peak hours (8–20). The demand array covers a subset of hours, so we use ``join="inner"`` to restrict the constraint to just those hours:

In [ ]:
peak_hours = pd.RangeIndex(8, 21, name="hour")
peak_demand = xr.DataArray(
    np.full(len(peak_hours), 120.0), dims=["hour"], coords={"hour": peak_hours}
)

total_gen = gen.sum("tech")
m3.add_constraints(total_gen.ge(peak_demand, join="inner"), name="peak_demand")

The demand constraint only applies during peak hours (8–20). Outside that range, no minimum generation is required.

## Summary

| Context | Default ``join`` | Behavior |
|---------|-----------------|----------|
| Arithmetic (``+``, ``-``, ``*``, ``/``) | ``"outer"`` | Union of coordinates; preserves associativity |
| Constraint with DataArray RHS | ``"left"`` | Expression defines positions; missing RHS → NaN (masked) |
| Constraint with expression RHS | ``"outer"`` | Goes through subtraction, inherits arithmetic default |

| ``join`` | Coordinates | Fill behavior |
|----------|------------|---------------|
| ``"outer"`` (arithmetic default) | Union | Fill with operation identity (0 for add, 0 for mul, 1 for div) |
| ``"inner"`` | Intersection only | No fill needed |
| ``"left"`` (constraint default) | Left operand's | Fill right with identity / NaN for constraint |
| ``"right"`` | Right operand's | Fill left with identity |
| ``"override"`` | Left operand's (positional) | Positional alignment, ignore labels |
| ``"exact"`` | Must match exactly | Raises error if different |